In [1]:
import polars as pl
import numpy as np

In [2]:
data = pl.read_csv("../data/cross_sell_email.csv")
data

gender,cross_sell_email,age,conversion
i64,str,i64,i64
0,"""short""",15,0
1,"""short""",27,0
1,"""long""",17,0
1,"""long""",34,0
1,"""no_email""",14,0
…,…,…,…
0,"""long""",18,0
1,"""no_email""",16,0
0,"""no_email""",15,0


In [3]:
data.group_by("cross_sell_email").agg(
    pl.col("gender", "age", "conversion").mean()
)

cross_sell_email,gender,age,conversion
str,f64,f64,f64
"""short""",0.633333,20.991667,0.125
"""long""",0.550459,21.752294,0.055046
"""no_email""",0.542553,20.489362,0.042553


In [8]:
import polars as pl

X = ["gender", "age"]

# Asumiendo que 'data' es tu DataFrame de Polars
result = (
    data.group_by("cross_sell_email")
    .agg([
        pl.col(c).mean().alias(f"{c}_mean") for c in X
    ] + [
        pl.col(c).var().alias(f"{c}_var") for c in X
    ])
    .with_columns([
        (
            (pl.col(f"{c}_mean") - pl.col(f"{c}_mean").filter(pl.col("cross_sell_email") == "no_email").first())
            /
            ((pl.col(f"{c}_var") + pl.col(f"{c}_var").filter(pl.col("cross_sell_email") == "no_email").first()) / 2).sqrt()
        ).alias(c) 
        for c in X
    ])
    .select(["cross_sell_email"] + X)
)

print(result)

shape: (3, 3)
┌──────────────────┬──────────┬──────────┐
│ cross_sell_email ┆ gender   ┆ age      │
│ ---              ┆ ---      ┆ ---      │
│ str              ┆ f64      ┆ f64      │
╞══════════════════╪══════════╪══════════╡
│ long             ┆ 0.015802 ┆ 0.221423 │
│ short            ┆ 0.184341 ┆ 0.08737  │
│ no_email         ┆ 0.0      ┆ 0.0      │
└──────────────────┴──────────┴──────────┘
